In [1]:
%cd ../

D:\Source\tbdx23


In [2]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sim.core.cascade import RepoCascade
from sim.core.inputs import load_inputs
from scipy.optimize import minimize_scalar
from sim.zaf.dhiv import ModelHIV
from scipy.interpolate import interp1d

In [3]:
iso = 'ZAF'

in_folder = f'results/C_{iso}'
out_folder = f'results/D_{iso}'

In [4]:
os.makedirs(out_folder, exist_ok=True)

In [5]:
pars_cs = pd.read_csv(f'results/pars_{iso}.csv')
repo_cs = RepoCascade(pars_cs, 2010)

inp = load_inputs(f'data/pars/{iso}')
model = ModelHIV(inp)

In [6]:
tarq = pd.read_csv(f'data/pars/{iso}/targets_q.csv').set_index('Time')
tarq

,iso,Country,Year,q,QCNR
Time,,,,,
2020.125,ZAF,South Africa,2020,1,0.001061
2020.375,ZAF,South Africa,2020,2,0.000647
2020.625,ZAF,South Africa,2020,3,0.000689
2020.875,ZAF,South Africa,2020,4,0.000809
2021.125,ZAF,South Africa,2021,1,0.000918
2021.375,ZAF,South Africa,2021,2,0.000556
2021.625,ZAF,South Africa,2021,3,0.000598
2021.875,ZAF,South Africa,2021,4,0.000720


In [7]:
particles = pd.read_csv(f'{in_folder}/Post.csv')
particles = [dict(row) for _, row in particles.iterrows()][:20]
for p in particles:
    p['cas'] = repo_cs.sample()
    p['irr_art'] = 0.3

In [8]:
sims = list()
for p in particles:
    p['cas'] = repo_cs.sample()
    
    ys, _, _ = model.simulate_to_preCOVID(p)
    sims.append((p, ys))


In [9]:
def fn(k, sims, tarq, t0, i0, t_in, y_in0):
    y_in0 = y_in0.copy()
    y_in0[i0] = k
    k_covid = interp1d(t_in, y_in0, kind='nearest', bounds_error=False, fill_value=1)

    ds = list()

    for p, ys in sims:
        p['k_covid'] = k_covid
        _, ms, _ = model.simulate_to_postCOVID(p, ys, t_eval=np.array([t0 - 1/8, t0 + 1/8]))
        d = (tarq.QCNR[t0] / ms.CNR - 1).pow(2).sum()
        ds.append(d)

    return sum(ds)


In [10]:
ts = list(tarq.index)
p, ys = sims[1]

t_in = np.array([2019.875] + ts)
y_in = np.ones_like(t_in)

sims0 = sims

for i, t in enumerate(t_in[2:], 2):
    if t > 2022:
        continue
    
    opt = minimize_scalar(fn, args=(sims0, tarq, t, i, t_in, y_in, ), method='bounded', bounds=[0.2, 5])
    
    # Update k_covid
    y_in[i] = opt.x
    k_covid = interp1d(t_in, y_in, kind='nearest', bounds_error=False, fill_value=1)

    sims1 = list()
    print(t, i, opt.x)
    for p, ys in sims0:
        p['k_covid'] = k_covid
        ys1, _, _ = model.simulate_to_postCOVID(p, ys, t_eval=np.array([t - 1/8, t + 1/8]))
        sims1.append((p, ys1))

    sims0 = sims1

2020.375 2 1.4212171714510384
2020.625 3 1.8972584668536367
2020.875 4 3.574474867150882
2021.125 5 4.999993745586689
2021.375 6 4.998356127027397
2021.625 7 4.999993411636527
2021.875 8 4.551474494968612


In [11]:
ts = list(tarq.index)
ts += [ts[-1] + 1 / 4]
ts = np.array(ts)

k_covid = interp1d(t_in, y_in, kind='nearest', bounds_error=False, fill_value=1)

mss = list()

for i, (p, ys) in enumerate(sims):
    p['k_covid'] = k_covid
    _, ms, _ = model.simulate_to_postCOVID(p, ys, t_eval=ts)
    mss.append(ms.assign(Key = i))

mss = pd.concat(mss)

In [12]:
pd.DataFrame({'Time': t_in, 'k_covid': y_in}).to_csv(f'{out_folder}/k_covid.csv')

In [13]:
mss.to_csv(f'{out_folder}/RunPost.csv')

In [14]:
pss = pd.DataFrame([p for p, _ in sims])
pss.to_csv(f'{out_folder}/Post.csv')